In [387]:
# This serves as a template which will guide you through the implementation of this task.  It is advised
# to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps
# First, we import necessary libraries:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF, Matern, RationalQuadratic
from sklearn.metrics import r2_score


In [388]:
def impute(data):
    #impute given data set with IterativeIputer function, not really sure how it works
    imp = IterativeImputer(random_state=0)
    imp.fit(data)
    imputed = imp.transform(data)
    return imputed

In [389]:

def data_loading():
    #true imputing after splitting x,y | false imputing before splitting
    ex = True
    """
    This function loads the training and test data, preprocesses it, removes the NaN values and interpolates the missing
    data using imputation

    Parameters
    ----------
    Returns
    ----------
    X_train: matrix of floats, training input with features
    y_train: array of floats, training output with labels
    X_test: matrix of floats: dim = (100, ?), test input with features
    """
    # Load training data


    train_df = pd.read_csv("train.csv")

    # print("Training data:")
    # print("Shape:", train_df.shape)
    # print(train_df.head(2))
    # print('\n')

    # Load test data
    test_df = pd.read_csv("test.csv")

    # print("Test data:")
    # print(test_df.shape)
    # print(test_df.head(2))

    #drop rows where y_train = NaN
    train_df = train_df.dropna(axis=0,subset='price_CHF')

    #change season string to int
    mapping = {'spring': 1, 'summer': 2, 'autumn': 3, 'winter': 4}
    train_df=train_df.replace({'season': mapping})
    test_df=test_df.replace({'season': mapping})

    #impute the training and test data This or impute X,y sets i dont know whats smarter
    if not ex:
         test_df = impute(test_df)
         train_df = impute(train_df)

    # Dummy initialization of the X_train, X_test and y_train

    if ex:
        X_train = train_df.drop(['price_CHF'],axis=1)
        y_train = train_df['price_CHF']
        X_test = test_df
    else:
        X_train = np.delete(train_df, 2, axis=1)
        y_train = train_df[:, 2]
        X_test = test_df



    #impute X's
    if ex:
        X_train = impute(X_train)
        X_test = impute(X_test)
    assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (X_test.shape[0] == 100), "Invalid data shape"
    return X_train, y_train, X_test

In [390]:

def modeling_and_prediction(X_train, y_train, X_test):
    """
    This function defines the model, fits training data and then does the prediction with the test data

    Parameters
    ----------
    X_train: matrix of floats, training input with 10 features
    y_train: array of floats, training output
    X_test: matrix of floats: dim = (100, ?), test input with 10 features

    Returns
    ----------
    y_test: array of floats: dim = (100,), predictions on test set
    """

    y_pred=np.zeros(X_test.shape[0])
    #TODO: Define the model and fit it using training data. Then, use test data to make predictions
    if False: #try inv mapping of seasons gaussian says no
        imapping = { 1:'spring',  2:'summer',  3:'autumn',  4:'winter'}
        X_train = [[imapping[num] if i == 0 else num for i, num in enumerate(row)]
                         for row in X_train]
        X_test = [[imapping[num] if i == 0 else num for i, num in enumerate(row)]
                         for row in X_test]

    if True:
        X_train = np.delete(X_train, 0, axis=1)
        y_train = y_train
        X_test = np.delete(X_test, 0, axis=1)


    KERNELS = [DotProduct, RBF, Matern, RationalQuadratic]
    gpr = GaussianProcessRegressor(kernel=Matern())
    gpr.fit(X_train, y_train)
    y_pred = gpr.predict(X_test)

    y_train_pred = gpr.predict(X_train)
    print(r2_score(y_train, y_train_pred))

    assert y_pred.shape == (100,), "Invalid data shape"
    return y_pred

In [391]:

# Main function. You don't have to change this
if __name__ == "__main__":
    # Data loading
    X_train, y_train, X_test = data_loading()
    # The function retrieving optimal LR parameters
    y_pred=modeling_and_prediction(X_train, y_train, X_test)
    # Save results in the required format
    dt = pd.DataFrame(y_pred)
    dt.columns = ['price_CHF']
    dt.to_csv('results_M4.csv', index=False)
    print("\nResults file successfully generated!")



1.0

Results file successfully generated!


In [0]:
# for name in list(train_df):
#    v = train_df[name].isna().sum()
#    print(f'{name}: {v}')
# msno.bar(train_df, figsize=(12, 6), fontsize=12, color='steelblue')
# season is complete rest of data is missing evenly, dropping empty rows results in len(train_df.dropna()) =0

In [395]:
train_df

,season,price_AUS,price_CHF,price_CZE,price_GER,price_ESP,price_FRA,price_UK,price_ITA,price_POL,price_SVK
0,1,NaN,9.644028,-1.686248,-1.748076,-3.666005,NaN,-1.822720,-3.931031,NaN,-3.238197
1,2,NaN,7.246061,-2.132377,-2.054363,-3.295697,-4.104759,-1.826021,NaN,NaN,-3.212894
2,3,-2.101937,7.620085,-1.910282,NaN,-3.388777,NaN,-2.034409,-4.073850,NaN,-3.114061
3,4,-2.098475,8.411894,-1.903834,NaN,-3.588235,NaN,-2.214720,-4.018620,-2.330803,NaN
4,1,-1.969687,8.926884,-1.697257,-1.331049,NaN,-3.911096,-2.388092,-4.093946,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
889,2,NaN,3.237347,0.235917,-0.306331,NaN,-2.735177,-1.839910,-2.378179,2.535228,NaN
890,3,-1.079859,2.679221,NaN,-0.226806,NaN,NaN,-1.807515,-2.223057,2.555153,2.189913
891,4,-0.961371,3.633928,0.352449,NaN,-3.488688,NaN,-1.568924,NaN,2.706716,2.520366
894,3,-1.186919,-0.303802,NaN,0.236214,-3.333672,-2.262145,-1.246151,-1.801203,NaN,NaN
